In [1]:
import importlib
import numpy as np
import pandas as pd
import utils
from glob import glob

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)
vocab_size

100

In [6]:
batch_size = 0

while batch_size < 700:
    train_filename = np.random.choice(glob('data/train_*.csv'), size=1)[0]
    before_scan = pd.read_csv(train_filename)
    batch_size = before_scan.shape[0]
batch_size

879

In [7]:
def scanning(array, input_length):
    batches = []
    text_length = array.shape[1]
    for i in range(text_length-input_length):
        batches.append(array[:, i:i+input_length+1])
    return np.concatenate(batches)

In [8]:
emb_dim = 128
input_length = 300

In [9]:
dataset = scanning(before_scan.values, input_length)

In [10]:
dataset.shape

(386760, 301)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=128, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=64, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (879, 300, 128)           12800     
_________________________________________________________________
conv1d_1 (Conv1D)            (879, 300, 256)           262400    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (879, 150, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (879, 150, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (879, 150, 128)           262272    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (879, 75, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (879, 75, 128)            0         
__________

In [13]:
model.set_weights(load_model('model_conv4_lstm2.h5').get_weights())

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
for _ in range(2):
    print(_)
    model.reset_states()
    model_hist = model.fit(dataset[:, :-1], dataset[:, [-1]], epochs=1, batch_size=batch_size)
    if model_hist.history['acc'][0] > .65: break

0
Epoch 1/1
386760/386760 [==============================] - 142s - loss: 1.3684 - acc: 0.5928   
1
Epoch 1/1
386760/386760 [==============================] - 140s - loss: 1.2354 - acc: 0.6329   


In [16]:
# save model
model.save('model_conv4_lstm2.h5')

In [17]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=8, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Dropout(.2))
pred_model.add(Conv1D(filters=128, kernel_size=8, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Dropout(.2))
pred_model.add(Conv1D(filters=64, kernel_size=8, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Dropout(.2))
pred_model.add(Conv1D(filters=32, kernel_size=8, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dropout(.2))
pred_model.add(Dense(vocab_size, activation='softmax'))

In [18]:
pred_model.set_weights(model.get_weights())

In [19]:
def implement(seed_text, maxlen=100, must_stop=2000, n_likely=5):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    pred_model.reset_states()
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-n_likely:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [20]:
res = implement('We are friends, good friends\nWe greet in the morning and say see you in the afternoon', 300, 600, 2)

In [21]:
print(res)

We are friends, good friends
We greet in the morning and say see you in the afternoon a love too do the world
I'm seet me and straight
The sunny all a looked to my heart don't say and she seen
That's  
I was they see that you don't say and stay
We're a little so soul that to say and show the sunshance
I can't see to the strong and strong and the stars of too strance and should she's a little love
I want to look at me
I want me to me and day to stop to me
Anything the sunshine
I won't let let me to me anymore
I won't let sorrow of my head
I'll stay and the way you care too she want to stay too late
I want to say to say and the way
A little so will with this day and day I can sa
